# Desafio Gerado pelo chat GPT
> Objetivo:
>> Carregar a Planilha: Carregar a planilha usando pandas e openpyxl.
>> Análise dos Dados:
>> Calcular o total de vendas por mês.
>> Identificar o SKU mais vendido em cada mês.
>> Calcular a quantidade total vendida para cada SKU ao longo do período.
>> Exportar os resultados para uma nova planilha Excel

In [2]:
import pandas as pd
import openpyxl 
import numpy as np

In [6]:
# lendo o arquivo  com o pandas

base_vendas_df = pd.read_excel('Base Vendas - 2022.xlsx')
base_vendas_df.head(10)

,Data da Venda,Ordem de Compra,SKU,ID Cliente,Qtd Vendida,ID Loja
0,2022-01-01,SO61285,HL216,23791,2,166
1,2022-01-01,SO61285,HL1,23791,1,93
2,2022-01-01,SO61285,HL227,23791,1,94
3,2022-01-01,SO61301,HL216,16747,2,17
4,2022-01-01,SO61301,HL89,16747,1,115
5,2022-01-01,SO61301,HL227,16747,1,95
6,2022-01-01,SO61269,HL2,11792,1,87
7,2022-01-01,SO61269,HL8,11792,1,183
8,2022-01-01,SO61286,HL215,11530,2,72
9,2022-01-01,SO61286,HL223,11530,2,131


In [12]:
# renomear as colunas 
base_vendas_df = base_vendas_df.rename(columns={'Data da Venda':'data_da_venda', 'Ordem de Compra': 'ordem_de_compra', 'ID Cliente':'id_cliente', 'Qtd Vendida': 'qtd_vendida', 'ID Loja':'id_loja'})
base_vendas_df.head()

,data_da_venda,ordem_de_compra,SKU,id_cliente,qtd_vendida,id_loja
0,2022-01-01,SO61285,HL216,23791,2,166
1,2022-01-01,SO61285,HL1,23791,1,93
2,2022-01-01,SO61285,HL227,23791,1,94
3,2022-01-01,SO61301,HL216,16747,2,17
4,2022-01-01,SO61301,HL89,16747,1,115


In [19]:
# Resolver o problema da data
vendas_janeiro_df = base_vendas_df.query('data_da_venda == "2022-01-20"')
vendas_janeiro_df.shape

(195, 6)